# Mobile robots project

<div style="border: 2px solid #4CAF50; border-radius: 10px; padding: 15px; background-color: #E8F5E9; color: #333;">
    <strong>T30 Members:</strong>
    <ul>
        <li>Jiwon You</li>
        <li>Adriana Nancy Orellana Torrico</li>
        <li>Charles Froessel</li>
        <li>Johann Lugon-Moulin</li>
</div>



-------

# Project description

## 1. Environment

- A good feature needs to be easy to extract and easy to match, robust to change in illumination, camera viewpoint and highly distinctive.
- From 3D to 2D: perspective projection
- Gaussian smoothing: removes high-frequency noise from the image with a gaussian kernel G
- Edge detection: canny edge filter (combination of smoothing + edge detector into one operator) is an optimal detector, to reduce its cost we can numerically approximate its behavior computing the edge direction and magnitude with Sobel.
- Sobel filter: from a row image convert to gray scale, then filter the image with gaussian or sobel filter, apply thresholding to get the edges, and finally non-maximum suppression to get the final edges.


## 2. Components

*TODO:* For each section, please explain how your imlementation works and how you have tested it. You can add diagrams, images or videos to make it easier to understand and show that your components are working as expected. Don't forget to cite external sources if you used any (even course materials).

### 2.1. Computer vision and image processing

**Responsible:** Adriana Nancy Orellana Torrico

**Slide intructions:** Your environment has to contain a set of obstacles that the Thymio avoids through global navigation. That is to say, the Thymio should avoid these obstacles without using the sensors to detect them. (remove when completed)

### 2.2. Path planning

**Responsible:** Charles Froessel

**Slides intructions:** The objective is that the Thymio goes from an arbitrary position in the map to a target that can be placed anywhere in the environment. These will be changed during the demo to see how your system performs. (remove when completed) 

**Link for schematics (Use epfl account):** https://epflch-my.sharepoint.com/:p:/r/personal/charles_froessel_epfl_ch/_layouts/15/doc.aspx?sourcedoc=%7B20d9373d-95f6-4339-9879-352d763cab56%7D&action=edit 

### 2.3. Pose estimation

**Responsible:** Jiwon You

#### 2.3.1 System Identification of a differetial drive robot
Below are the names to denote state/control variables of Thymio. \
\
$u_l$ = left motor control input, `motor.left.target` \
$u_r$ = right motor control input, `motor.right.target` \
$u_{l,m}$ = measured left motor speed, `motor.left.speed` \
$u_{r,m}$ = measured left motor speed, `motor.right.speed` \
$v_{l}$ = linear velocity of left wheel \
$v_{r}$ = linear velocity of right wheel \
$v$ = linear velocity of Thymio \
$w$ = angular velocity of Thymio 

First, in `calibration.ipynb` I ran calibration code from exercise 8 to discover relationship between motor speed and linear velocity of Thymio. We will mainly use $u_l$ and $u_r$ around 100 to control Thymio, so we investigate linear velocity and motor speed variance at $u=100$. Moreover, we assume that conversion between motor speed and linear velocity is linear around this point. \
![My Image](figure/ground_sensor.png) \
Knowing that there is a thin white stripe between black blocks every 5cm, we can detect peaks from ground sensor and compute linear speed from the time it took to cross 6 blocks.\
It returned linear velocity = `3.26cm/s` for motor speed 100, and thus conversion factor from motor speed to linear velocity was `0.0326`. \
![My Image](figure/target_and_measured_speed_of_motor.png)\
In the histogram below, measured motor speed forms a Gaussian distribution around motor target speed of 100.\
![My Image](figure/measured_motor_speed_histogram.png)

With $\alpha$ as conversion factor and $L$ as Thymio wheel axle length,  \
$v_l = \alpha u_l$ \
$v_r = \alpha u_r$ \
$v = \frac{v_l + v_r}{2}$ \
$w = \frac{v_r - v_l}{L}$ \
Conversion between $v,w$ and $u_l, u_r$ is implemented in `utils.py`.
```py
    wheel_axle_length = 9.5 # in cm
    thymio_speed_to_cms = 0.03260
    def from_u_to_vw(ul, ur):
        thymio_speed_to_cms = 0.03260
        A = thymio_speed_to_cms* np.array([[0.5, 0.5],[-1/wheel_axle_length, 1/wheel_axle_length]])
        # vl = thymio_speed_to_cms * ul
        # vr = thymio_speed_to_cms * ur
        # v = (vl + vr) / 2
        # w = (vr - vl) / wheel_axle_length
        vw = A@np.array([ul, ur]) 
        return vw[0], vw[1] # returns v, w in cm/s, rad/s

    def from_vw_to_u(v,w):
        A = thymio_speed_to_cms* np.array([[0.5, 0.5],[-1/wheel_axle_length, 1/wheel_axle_length]])
        A_inv = np.linalg.inv(A)
        ulur = A_inv @ np.array([v,w]) 
        return int(ulur[0]), int(ulur[1]) # returns ul, ur as int
```
Let $X = (x,y,\theta, v, w)^{T}$ denote Thymio state. Discrete time motion model of Thymio is as follows.
$$
\begin{align*}
X_{i+1} 
&= f(X_i, u_i) \\
&=
\begin{bmatrix}
x_i + v_i\cdot cos(\theta_i)\Delta t\\
y_i + v_i\cdot sin(\theta_i)\Delta t\\
\theta_i + w_i\cdot\Delta t \\
v_{i+1}\\
w_{i+1}
\end{bmatrix} \\
(v_{i+1}, w_{i+1}) &= from\_u\_to\_vw(u_i)
\end{align*}
$$ //
This is implemented in `ExtendedKalmanFilter.move(u)` of `kalman.py`.\
```py
    def move(self, u):
        assert(u.shape[0] == 2) # u is np.array containing v_,w_
        x,y,theta,v,w = self.X
        v_, w_ = u
        self.X[0] = x + v*np.cos(theta)*self.dt 
        self.X[1] = y + v*np.sin(theta)*self.dt
        self.X[2] = self.clip_theta(theta + w * self.dt)
        self.X[3] = v_ 
        self.X[4] = w_
``` 

#### 2.3.2 Process and measurement noise
For Kalman Filter, let Q denote process noise matrix, R measurement noise matrix, $R_c$ camera noise matrix, $R_{vw}$ linear/angular velocity measurement noise matrix. Since we can't really estimate process noise, I used empirical tuning for Q. 
$$
R = 
\begin{bmatrix}
R_c & 0 \\
0 & R_{vw}
\end{bmatrix}
$$
Measurement noise is split into camera noise and vw noise. It is safe to assume that camera noise and vw noise are uncorrelated, and thus R is in block diagonal form. \
Camera measurements are faily accurate, and we can use a small value for camera noise. However, when aruco marker on Thymio is blocked, camera cannot detect Thymio. For such case, camera will return $(x_{cam}, y_{cam}, \theta_{cam}) = (0, 0, 0)$ and camera noise will be set to infinity, which excludes camera measurement from updating Thymio state. This is implemented in `ExtendedKalmanFilter.switch_mode()` of `kalman.py`.\
```py
    NOISE_COV_VW = np.array([[0.01285577, 0.00072932],
                    [0.00072932, 0.00056978]]) # from calibration.ipynb
    NOISE_COV_CAMERA = 0.0001*np.eye(3)
    NOISE_COV_CAMERA_BLOCKED=9999999*np.eye(3)
    PROCESS_COV = 0.01*np.eye(5)
    def switch_mode(self, camera_blocked):
        if camera_blocked and self.camera_available:
            self.R[0:3, 0:3] = NOISE_COV_CAMERA_BLOCKED
            self.camera_available = False
        elif not camera_blocked and not self.camera_available:
            self.R[0:3,0:3] = NOISE_COV_CAMERA
            self.camera_available = True
``` 
We have previously seen that motor speed measurement noise resembles Gaussian. Since we have direct access to motor speed measurement, we can easily compute motor speed variance. We assume that noise from left and right motor are uncorrelated.\
![My Image](figure/target_and_measured_speed_of_motor.png)\
Then using linear transformation, we can convert motor speed variance to linear/angular velocity variance. As in 2.3.1, $\alpha$ denotes Thymio speed conversion factor.
$$
\begin{align*}
\begin{bmatrix}
v \\
w
\end{bmatrix}
&=
\begin{bmatrix}
\frac{\alpha}{2} & \frac{\alpha}{2} \\
\frac{-\alpha}{L} & \frac{\alpha}{L} 
\end{bmatrix} \cdot
\begin{bmatrix}
u_l \\
u_r
\end{bmatrix} = A\cdot u \\
\Sigma_{u} &= 
\begin{bmatrix}
\sigma_{ul}^2 & 0 \\
0 & \sigma_{ur}^2
\end{bmatrix} \\
\Sigma_{vw} &= A\cdot\Sigma_{u}\cdot A^T
\end{align*}
$$ //
```py
    var_l_speed = np.var(l_speed[idx:]) # took from idx to avoid the initial transient and devid by the conversion factor
    var_r_speed = np.var(r_speed[idx:])
    wheel_axle_length = 9.5 # in cm
    A = thymio_speed_to_cms* np.array([[0.5, 0.5],[-1/wheel_axle_length, 1/wheel_axle_length]]) # (v,w) = A @ (ul, ur)
    cov_vw = A @ np.array([[var_l_speed,0],[0, var_r_speed]]) @ A.T
```
$$
R_{vw} =
\begin{bmatrix}
0.01286 & 0.00073 \\
0.00073 & 0.00057
\end{bmatrix}
$$
Note that $R_{vw}$ is not diagonal. This agrees with our intuition that linear/angular velocity noise are correlated.
#### 2.3.3 Extended Kalman Filter implementation
Since our Thymio is a time-variant nonlinear system and noises can be assumed as Gaussian, we are using Extended Kalman Filter.
$$
\begin{align*}
X_{i+1} 
&= f(X_i, u_i) \\
&=
\begin{bmatrix}
x_i + v_i\cdot cos(\theta_i)\Delta t\\
y_i + v_i\cdot sin(\theta_i)\Delta t\\
\theta_i + w_i\cdot\Delta t \\
v_{i+1}\\
w_{i+1}
\end{bmatrix} \\
y &= h(X) = X
\end{align*}
$$ 
First we compute Jacobian of $f(X_i, u_i)$ to linearize our system.
$$
\begin{align*}
F
&= \frac{\partial f(X,u)}{\partial X} \bigg|_{X_i, u_i}
&=
\begin{bmatrix}
1 & 0 & -sin(\theta)v\Delta t & cos(\theta)\Delta t & 0 \\
0 & 1 & cos(\theta)v\Delta t & sin(\theta)\Delta t & 0 \\
0 & 0 & 1 & 0 & \Delta t \\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0
\end{bmatrix} 
\end{align*}
$$
Predict state covariance
$$
P = FPF^{T}+Q
$$
Predict state
$$
\bar X = f(X,u)
$$
Measurement residual. For our case, measurement funtion $h(X)$ is an identity function that returns $X$, and thus Jacobian of $h(X)$ is an identity matrix.
$$
y = z - h(X) = z - X
$$
$$
H = \frac{\partial h(X)}{\partial X} = 
\begin{bmatrix}
1 & 0 & 0 & 0 & 0\\
0 & 1 & 0 & 0 & 0\\
0 & 0 & 1 & 0 & 0\\
0 & 0 & 0 & 1 & 0\\
0 & 0 & 0 & 0 & 1\\
\end{bmatrix}
$$
Residual covariance
$$
S = HPH^{T}+R
$$
Near optimal Kalman gain
$$
K = PH^{T}S^{-1}
$$
update state
$$
X = X + Ky
$$
update state covariance
$$
P = (I-KH)P
$$
This is implemented in `ExtendedKalmanFilter.predict_and_update()` of `kalman.py`.
```py
    def predict_and_update(self, u, z):
        assert(u.shape[0] == self.control_dim)
        assert(z.shape[0] == self.measurement_dim)
        F = self.compute_F()
        self.move(u)
        self.P = F @ self.P @ F.T + self.Q
        y = z - self.X # use measurement model H(X) = X
        y[2] = self.clip_theta(y[2])
        S = self.H @ self.P @ self.H.T + self.R
        S_inv = np.linalg.inv(S)
        K = self.P @ self.H.T @ S_inv
        self.X = self.X + K @ y
        self.P = (np.eye(self.state_dim) - K @ self.H) @ self.P
        self.state_trajectory.append(self.X)
``` 
Consider a case where $\theta_{i+1} = 0.01, \theta_i = 2\pi - 0.01$. Simple subtraction would yield 
$$ 
\Delta\theta = \theta_{i+1} - \theta = 0.02 - 2\pi
$$
, but the value should be just 0.02./
To prevent such overflow, we use `ExtendedKalmanFilter.clip_theta()` to keep $\theta$ values in the range of $[-\pi, \pi]$.
```py
    def clip_theta(self, theta):
        return (theta + np.pi) % (2 * np.pi) - np.pi
``` 
In `kalman.ipynb` I tested my implementation with a simple simulation under noise. Despite initial state with large error, EKF is able to converge to true state.
![My Image](figure/ekf_simulation.png)
#### References
Control your Thymio in Python, Basics of Mobile Robotics \
Solution 8, Basics of Mobile Robotics \
https://en.wikipedia.org/wiki/Extended_Kalman_filter \
https://github.com/rlabbe/filterpy 

##### Acknowledgement
overall structure of ExtendedKalmanFilter class was inspired by https://github.com/grilloandrea6/mobile-robotics-project 

### 2.4. Obstacle avoidance

**Responsible:** Johann Lugon-Moulin

**Slides intructions:** While navigating, the Thymio will have to use local navigation to avoid physical obstacles that can be put in its path at any point in time. You are free to choose what these physical objects are. (remove when completed)

## 3. Running the project

*TODO:* Add instructions on how to run the project, and show a video of the project working.

## 4. Conclusion